# Analisis de Egresos Hospitalarios

In [1]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import lectura_archivos as fr
import calculo_metricas as aux

from functools import reduce

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(fr)
importlib.reload(aux)


<module 'calculo_metricas' from 'c:\\Users\\ppizarro\\Desktop\\Javier_Rojas\\Javier\\trabajo\\SSMO\\EPH\\1.0-modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\calculo_metricas.py'>

In [2]:
HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax


## Lectura de archivos

- Se leen todos los archivos que estan en input/utf-8. Los archivos estan separados por ;
- Se dejan solamente los egresos de los diagnosticos que **SI** tenga el Tórax
- Se cambia la codificacion de INTERV_Q de 2 (No, 1 es SI) a 0.
- Se cambia la codificacion de CONDICION_EGRESO (orig 1 = Vivo, 2 = Fallecido) de 1 -> 0 y 2 -> 1
- Se cambia la codificacion de GLOSA_REGION_RESIDENCIA {
                        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                    }

In [3]:
df = fr.leer_archivos(HOSPITAL_A_ANALIZAR)

In [4]:
aorticos = [
    "I06X",
    "I060",
    "I061",
    "I062",
    "I068",
    "I069",
    "I080",
    "I082",
    "I083",
    "I35X",
    "I350",
    "I351",
    "I352",
    "I358",
    "I359",
    "I391",
    "Q23X",
    "Q230",
    "Q231",
    "Q238",
    "Q239",
    "Q244",
]

In [5]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

df_nacional = pl.scan_csv("input/utf-8/*.csv", separator=";")
df_nacional = fr.mappear_columnas(df_nacional, fr.MAPPING_METRICAS_EGRESOS)
metricas = aux.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()
cie = fr.leer_cie()

In [6]:
metricas_con_cie = metricas.join(cie, on="DIAG1", how="left")
aorticas = metricas_con_cie.filter(pl.col("Descripción").str.contains("aórtica"))

# Analisis de Egresos Hospitalarios

Se calculan las siguientes metricas:

- Número de Egresos (n_egresos)
- Dias estada totales (dias_estada_totales)
- Cantidad de Intervenciones Quirúrgicas (n_int_q)
- Cantidad de Muertes (n_muertos)

Todas estas metricas estan calculadas a nivel de Año, Establecimiento de Salud y Diagnóstico. Por
ejemplo:

***"En el año 2019, el Hospital San José (código XXXXXXX) tuvo XX egresos, con XX dias estada_promedio,
con XX intervenciones_quirúrgicas y XX muertos para el diagnóstico A24.4."***

Este análisis se realizó entre 2001 y 2020 para todos los hospitales/clínicas de Chile.

In [7]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = aux.obtener_metricas_egresos(df, AGRUPACION).collect(streaming=True)


# Analisis de Estratos

In [8]:
dict_estratos = aux.obtener_diccionario_estratos(df, HOSPITAL_A_ANALIZAR)
cie = fr.leer_cie()


In [22]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION

ORDEN_COLS = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "Capítulo",
    "Sección",
    "Categoría",
    "Descripción",
    "DIAG1",
    "n_egresos",
    "dias_estada_totales",
    "n_int_q",
    "n_muertos",
    "ranking_nacionales_n_egresos",
    "total_nacionales_n_egresos",
    "%_nacionales_n_egresos",
    "ranking_publicos_n_egresos",
    "total_publicos_n_egresos",
    "%_publicos_n_egresos",
    "ranking_privados_n_egresos",
    "total_privados_n_egresos",
    "%_privados_n_egresos",
    "ranking_grd_n_egresos",
    "total_grd_n_egresos",
    "%_grd_n_egresos",
    "ranking_interno_n_egresos",
    "total_interno_n_egresos",
    "%_interno_n_egresos",
]


In [23]:
dict_resultados_estratos = aux.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

resultado = reduce(aux.left_join_consecutivo, dict_resultados_estratos.values())
resultado_unido = resultado.join(cie, how="left", on="DIAG1")
resultado_unido = resultado_unido.select(ORDEN_COLS)
resultado_unido_pandas = resultado_unido.to_pandas()

In [21]:
resultado_unido_pandas.to_csv(
    "output/ranking_hospitales_maestro/rankings_hospitales_maestro.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

resultado_unido_pandas.query("ANO_EGRESO > 2002").to_excel(
    "output/ranking_hospitales_maestro/rankings_hospitales_maestro.xlsx", index=False
)

In [19]:
resultados_torax = resultado_unido.filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
torax = resultados_torax.to_pandas()
torax.to_csv(
    "output/ranking_torax_pais/resumen_posiciones_torax_pais.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)


## Obtencion de 20 diagnosticos mas relevantes para el Torax por produccion

Para obtener los diagnosticos mas relevantes para el torax productivamente se siguieron los
siguientes pasos:

1. Se dejaron solamente los diagnosticos que alguna vez hayan estado primeros en el ranking
nacional segun produccion.
2. Luego, se dejaron los que tuvieran mas de 0 intervenciones quirurgicas y que hayan
aparecido luego del anio 2011.
3. De los diagnosticos restantes, por anio se seleccionaron los 20 que tuvieran la mayor
cantida de egresos.

In [16]:
diags_mas_relevantes = resultados_torax.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") > 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio = diags_mas_relevantes.groupby(["ANO_EGRESO"]).head(20)
DIAGS_MAS_RELEVANTES = veinte_mas_relevantes_por_anio.select(pl.col("DIAG1").unique()).sort(
    by="DIAG1"
)


In [17]:
print(f"Los diagnosticos mas relevantes para el Torax son: {DIAGS_MAS_RELEVANTES}")

veinte_mas_relevantes_por_anio.write_excel("output/ranking_torax_pais/mas_relevantes.xlsx")


Los diagnosticos mas relevantes para el Torax son: shape: (46, 1)
┌───────┐
│ DIAG1 │
│ ---   │
│ str   │
╞═══════╡
│ C33X  │
│ C340  │
│ C341  │
│ C342  │
│ …     │
│ Q676  │
│ T820  │
│ T821  │
│ Z450  │
└───────┘


## Obtencion de resultados para diagnosticos de congenitas

In [21]:
GLOSA_CONGENITAS = (
    "Cap.17  MALFORMACIONES CONGÉNITAS, DEFORMIDADES Y ANOMALÍAS CROMOSÓMICAS (Q00-Q99)"
)

resultados_congenitas = resultados_torax.filter(pl.col("Capítulo") == GLOSA_CONGENITAS).sort(
    by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True
)

resultados_congenitas.to_pandas().to_csv(
    "output/ranking_torax_pais/resumen_egresos_congenitas.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)


# Analisis Sociodemográfico

In [34]:
desglose_sociodemografico = [
    "ANO_EGRESO",
    "REGION_RESIDENCIA",
    "GLOSA_REGION_RESIDENCIA",
    "region_pais",
    "COMUNA_RESIDENCIA",
    "GLOSA_COMUNA_RESIDENCIA",
    "comuna_region_pais",
    "DIAG1",
]

variables_a_analizar = [
    "DIAG1",
    "SEXO",
    "EDAD_CATEGORIA",
    "PUEBLO_ORIGINARIO",
    "GLOSA_PAIS_ORIGEN",
    "PREVISION",
    "BENEFICIARIO",
]

In [35]:
df_socio = fr.obtener_df_inicial_sociodemografico()


In [36]:
dict_resultados = {}

for variable in variables_a_analizar:
    agrupacion_total = desglose_sociodemografico.copy()
    if variable != "DIAG1":
        agrupacion_total += [variable]

    resultado = (
        df_socio.groupby(agrupacion_total)
        .agg(pl.col(variable).count().alias("conteo"))
        .sort(["ANO_EGRESO", "DIAG1", "conteo"], descending=True)
        .with_columns(
            pl.concat_str(pl.col(desglose_sociodemografico), separator="-").alias("llave_id")
        )
    )

    dict_resultados[variable] = resultado


In [37]:
import xlsxwriter

with xlsxwriter.Workbook("output/distribucion_sociodemografica/distribucion_polars.xlsx") as f:
    for var, df in dict_resultados.items():
        df.write_excel(workbook=f, worksheet=var)


# Analisis de cantidad de Camas a ocupar

- Aqui se quiere ver cuantas camas son necesarias por diagnostico
- El periodo que se quiere obtener es cada mes.
- Por lo tanto, se deben obtener la cantidad de dias estada totales por mes. Luego, dividir
esa cantidad por los dias del periodo.

In [24]:
AGRUPACION_MENSUAL = [
    "ANO_EGRESO",
    "MES_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

In [25]:
with pl.StringCache():
    metricas_mensuales = aux.obtener_metricas_egresos(df, AGRUPACION_MENSUAL).collect(
        streaming=True
    )
    metricas_mensuales_torax = metricas_mensuales.filter(pl.col("ESTABLECIMIENTO_SALUD") == 112103)
    metricas_mensuales_torax = metricas_mensuales_torax.join(cie, how="left", on="DIAG1")
    metricas_mensuales_torax.to_pandas().to_csv(
        "output/cantidad_camas/resumen_torax_resolucion_mensual.csv",
        sep=";",
        decimal=".",
        encoding="latin-1",
        index=False,
    )


ColumnNotFoundError: MES_EGRESO

Error originated just after this operation:
 WITH_COLUMNS:
 [col("CONDICION_EGRESO").map()]
   WITH_COLUMNS:
   [col("INTERV_Q").map()]
    FILTER col("DIAG1").is_in([Series[DIAG1].alias("DIAG1")]) FROM
    RECHUNK
      UNION:
      PLAN 0:

          CSV SCAN input\utf-8\Egresos_Hospitalarios_2001.csv
          PROJECT */37 COLUMNS
      PLAN 1:

          CSV SCAN input\utf-8\Egresos_Hospitalarios_2002.csv
          PROJECT */37 COLUMNS
      PLAN 2:

          CSV SCAN input\utf-8\Egresos_Hospitalarios_2003.csv
          PROJECT */37 COLUMNS
      PLAN 3:

          CSV SCAN input\utf-8\Egresos_Hospitalarios_2004.csv
          PROJECT */37 COLUMNS
      PLAN 4:

          CSV SCAN input\utf-8\Egresos_Hospitalarios_2005.csv
          PROJECT */37 COLUMNS
      PLAN 5:

          CSV SCAN input\utf-8\Egresos_Hospitalarios_2006.csv
          PROJECT */37 COLUMNS
      PLAN 6:

          CSV SCAN input\utf-8\Egresos_Hospitalarios_2007.csv
          PROJ